In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt 

ACTIVATION FUNCTIONS

In [3]:

##1.Sigmoid Activation Function 
def sig(z):
    s=1/(1+np.exp(-z))
    return s

def dsig(s):
    das=s*(1-s)
    return das


##TANH ACTIVATION FUNCTION
def tanh(z):
    s= (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))
    return s

def dtanh(s):
    dat=(1-s**2)
    return dat


##Relu activation function 
def relu(z):
    s=np.maximum(0,z)
    return s 

def drelu(s):
    dar=(np.int64(s>0))
    return dar

##Leaky Relu 
def lrelu(z):
    s=np.maximum(0.01*z,z)     ## we can also take 0.01
    return s 

def dlrelu(s,alpha=0.01):
    dal=(np.where(s>0,1,alpha))
    return dal


##Softmax (similar as sigmoid , with multiple dimension and used for multiclass)
def softmax(z):
    e = np.exp(z)
    s=e/np.sum(e)
    return s


In [4]:

X=np.random.randn(2,3)
print(X)
y = (np.random.randn(1,3)>0)   ## if > 0 its true or else its false
print(y)
y.shape[1]

[[ 0.89210364  0.2812165  -1.0763155 ]
 [-0.25434658 -0.14203968 -2.73456167]]
[[False  True False]]


3

FORWARD PROPAGATION

In [5]:
##Forward Propagation

##DEFINE THE SHAPE 
def layers_size(X,y,node):
    n_x=X.shape[0]
    n_h= node
    n_y=y.shape[0]
    return n_x,n_y,n_h

np.random.seed(2)

#initialization 
def intz(n_x,n_y,n_h):
    w1=np.random.randn(n_h,n_x)*0.01   ##0.01 value chota krne liya hai , can be ignored 
    b1= np.zeros((n_h,1))
    w2=np.random.randn(n_y,n_h)*0.01
    b2= np.zeros((n_y,1))   
    
    parameters = {"w1":w1,"b1":b1,"w2":w2,"b2":b2}   ##for ease 
    return parameters 


##Forward Propogation 
def fwd(X,parameters):
    w1=parameters["w1"]
    b1=parameters["b1"]
    w2=parameters["w2"]
    b2=parameters["b2"]
    
    z1= np.dot(w1,X)+b1
    a1=tanh(z1)  ## np.tanh
    z2=np.dot(w2,a1)+b2
    a2=np.array(sig(z2))
    
    cache={"z1":z1,"a1":a1,"z2":z2,"a2":a2}
    return a2,cache 

##COST 
def compute_cost(a2,y):
    m= y.shape[1]
    logp =np.multiply(np.log(a2),y)+np.multiply(np.log(1-a2),(1-y))
    cost = -np.sum(logp)/m
    cost = float(np.squeeze(cost))    ## optional 
    return cost 
    


In [6]:
a,b,c = layers_size(X,y,4)  ##4 nodes

In [7]:
parameters = intz(a,b,c)
parameters

{'w1': array([[-0.00416758, -0.00056267],
        [-0.02136196,  0.01640271],
        [-0.01793436, -0.00841747],
        [ 0.00502881, -0.01245288]]),
 'b1': array([[0.],
        [0.],
        [0.],
        [0.]]),
 'w2': array([[-0.01057952, -0.00909008,  0.00551454,  0.02292208]]),
 'b2': array([[0.]])}

In [8]:
a2 , cache = fwd(X,parameters)
cache

{'z1': array([[-0.0035748 , -0.00109207,  0.00602428],
        [-0.02322906, -0.00833717, -0.02186201],
        [-0.01385835, -0.00384782,  0.04232113],
        [ 0.00765357,  0.00318299,  0.02864058]]),
 'a1': array([[-0.00357478, -0.00109207,  0.00602421],
        [-0.02322488, -0.00833698, -0.02185852],
        [-0.01385746, -0.0038478 ,  0.04229588],
        [ 0.00765342,  0.00318298,  0.02863275]]),
 'z2': array([[0.00034795, 0.00013908, 0.00102453]]),
 'a2': array([[0.50008699, 0.50003477, 0.50025613]])}

In [9]:
cost =compute_cost(a2,y)
cost

0.6933527965194051

BACK PROPAGATION 

In [10]:
#Backward Propagation
def BWP(parameters,X,y,cache):
    w1=parameters["w1"]
    b1=parameters["b1"]
    w2=parameters["w2"]
    b2=parameters["b2"]
    
    a1= cache["a1"]
    a2= cache["a2"]
    
    m=y.shape[1]  ##y.size
    
    dz2=a2-y
    dw2 = np.dot(dz2,a1.T)/m
    db2=np.sum(dz2,axis=1,keepdims=True)/m
    da1= np.dot(w2.T,dz2)
    dz1= da1*(1-a1**2) ##np.power(a1,2) , dtanh(a1)
    dw1=np.dot(dz1,X.T)/m
    db1=np.sum(dz1,axis=1,keepdims=True)/m
    
    grades = {"dw1":dw1,"db1":db1,"dw2":dw2,"db2":db2}
    return grades

#7.Update Grades
def update(parameters,grades,lr=0.01):
    w1=parameters["w1"]
    b1=parameters["b1"]
    w2=parameters["w2"]
    b2=parameters["b2"]

    dw1=grades["dw1"]
    db1=grades["db1"]
    dw2=grades["dw2"]
    db2=grades["db2"]


    w1=w1-lr*dw1
    b1=b1-lr*db1
    w2=w2-lr*dw2
    b2=b2-lr*db2

    parameters={"w1":w1,"b1":b1,"w2":w2,"b2":b2}

    return parameters

In [11]:
grades=BWP(parameters,X,y,cache)
grades

{'dw1': array([[ 0.00082128,  0.00502214],
        [ 0.00070562,  0.00431307],
        [-0.0004265 , -0.00261332],
        [-0.00177635, -0.01087296]]),
 'db1': array([[-0.0017645 ],
        [-0.00151472],
        [ 0.00091798],
        [ 0.00381992]]),
 'dw2': array([[ 0.00059065, -0.00612704,  0.0053842 ,  0.0055199 ]]),
 'db2': array([[0.16679263]])}

In [12]:
parameters=update(parameters,grades,lr=0.01)
print(parameters)

{'w1': array([[-0.00417579, -0.00061289],
       [-0.02136902,  0.01635958],
       [-0.01793009, -0.00839134],
       [ 0.00504658, -0.01234415]]), 'b1': array([[ 1.76450187e-05],
       [ 1.51472143e-05],
       [-9.17976746e-06],
       [-3.81991525e-05]]), 'w2': array([[-0.01058543, -0.00902881,  0.0054607 ,  0.02286688]]), 'b2': array([[-0.00166793]])}


In [13]:
##CREATE OWN NN
def NN(X,y,itr=10000,print_cost=True,layers=4):
    np.random.seed(3)
    n_x,n_h,n_y=layers_size(X, y,layers)
    parameters=intz(n_x, n_h, n_y)

    for i in range (itr):
        a2, cache = fwd(X, parameters)
        cost=compute_cost(a2, y)
        grades=BWP(parameters,X,y,cache)
        parameters=update(parameters,grades,lr=0.01)

        if print_cost and i%100==0:
            print(f"cost{i}: {cost}")
            
    return parameters

In [14]:
NN(X,y)

cost0: 0.6931088338321277
cost100: 0.671042850253247
cost200: 0.65743313529991
cost300: 0.6486991062747438
cost400: 0.6425295126261946
cost500: 0.6372822237842959
cost600: 0.6316686686419338
cost700: 0.6247302772848742
cost800: 0.616023876465022
cost900: 0.6056916943941218
cost1000: 0.5941914098971292
cost1100: 0.5819446472745855
cost1200: 0.5692054882660894
cost1300: 0.5561010540732704
cost1400: 0.5426876833129942
cost1500: 0.528967470846709
cost1600: 0.514882754938318
cost1700: 0.5003160741180636
cost1800: 0.4851085061118905
cost1900: 0.4690931879270996
cost2000: 0.4521323892386781
cost2100: 0.4341471303710605
cost2200: 0.41513405333275966
cost2300: 0.39516995658136916
cost2400: 0.37440712193701714
cost2500: 0.353062298587842
cost2600: 0.3314008537814525
cost2700: 0.3097169808106262
cost2800: 0.2883114195527962
cost2900: 0.2674691468675357
cost3000: 0.24743999576658915
cost3100: 0.2284247336422297
cost3200: 0.2105679368810038
cost3300: 0.19395754809399948
cost3400: 0.1786298150311593

{'w1': array([[-0.9857339 ,  0.97572892],
        [ 1.86351992, -1.60834794],
        [ 1.64924855, -1.45093869],
        [-0.49067231,  0.59699412]]),
 'b1': array([[ 0.73566991],
        [-1.3871144 ],
        [-1.2284607 ],
        [ 0.34460333]]),
 'w2': array([[ 1.67319922, -3.13995004, -2.76007115,  0.88630804]]),
 'b2': array([[0.0752462]])}